In [1]:

import matplotlib.pyplot as plt
import modules.cosmos_functions as cf
import numpy as np
import pprint
import torch
import torch.nn.functional as F
import torchvision.transforms as T


from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


In [2]:
# functions to display images
def reverse_normalize(image):
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = image.clone()
    for i in range(3):
        image[i] = (image[i] * std[i]) + mean[i]
    return image

def show_batch(test_d):
    # Get the first batch of data from the DataLoader
    data_test = next(iter(test_d))

    # Retrieve the first tensor and its corresponding label
    image_test = data_test[0][0]
    label_test = data_test[1][0]

    # Reverse the normalization of the image
    image_test = reverse_normalize(image_test)

    # Convert the image tensor to a NumPy array and transpose the dimensions
    np_image_test = image_test.permute(1, 2, 0).numpy()

    # Display the image
    plt.imshow(np_image_test)
    plt.title(f'{label_test}, {image_test.shape}')
    plt.axis('off')

    # Show the plot
    plt.show()


In [3]:
# set the device

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
x = torch.ones(1, device=device)

print(f"Device is '{device}' Thus a tensor will look like this: {x}")

Device is 'mps' Thus a tensor will look like this: tensor([1.], device='mps:0')


In [4]:
# add the resnet18 model for testing the test dataset with the trained model
# import the resnet18 model
from torchvision import models
model = models.resnet18(pretrained=True)

# freeze the model parameters
for param in model.parameters():
    param.requires_grad = False
    
# change the last layer of the model to fit the number of classes in the dataset
model.fc = nn.Linear(512, 4)



/Users/stephandekker/.local/share/virtualenvs/pink_lady-bMzUwwsu/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/stephandekker/.local/share/virtualenvs/pink_lady-bMzUwwsu/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
class CNN(nn.Module):
    def __init__(self, dropout=0):
        super(CNN, self).__init__()
        self.dropout = dropout
        
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        
        # Calculate the output size after pooling and convolutions
        self.fc_input_size = 16 * 53 * 53  # Adjusted for input size 224x224
        
        self.fc1 = nn.Linear(self.fc_input_size, 120)
        self.dropout1 = nn.Dropout(self.dropout)
        self.fc2 = nn.Linear(120, 84)
        self.dropout2 = nn.Dropout(self.dropout)
        self.fc3 = nn.Linear(84,4)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1, self.fc_input_size)  # Flatten the output of the conv layer automatically
        
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        # x = F.softmax(self.fc3(x), dim=1)  # softmax on dim 1 to get probabilities for apple
        
        return x


In [6]:
# # Show the 1st img in the dataset
# show_batch(test_d)
# print(test_dataset[0][0])

In [7]:
imported_model_path = "../storage/data/generated/20230608-100518_pinky_acc.pt"   # test to test 224x224
# imported_model_path = "../storage/data/generated/20230605-134750_pinky_acc.pt"  # high accuracy
# imported_model_path = cf.load_pth('20230605_160852_pinky')  # issues; WIP
imported_model_path 


'../storage/data/generated/20230608-100518_pinky_acc.pt'

In [19]:


# Load the model
# model = CNN()
model_import_path = imported_model_path
model.load_state_dict(torch.load(model_import_path))
model.to(device)

# Load the test dataset
dataset_path = "../storage/images/apple_disease_classification/Test"

# Define the label dictionary
labels_dict = {
    'Blotch_Apple': 0,
    'Normal_Apple': 1,
    'Rot_Apple': 2,
    'Scab_Apple': 3
}

def test_model(model, datasetPath):
    model.eval()    

    # Load the test dataset
    dataset_path = datasetPath
    transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
    dataset = ImageFolder(dataset_path, transform=transform)
    test_dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

    labels_dict = dataset.class_to_idx

    # Track the overall test accuracy and accuracy by each type of apple
    overall_correct = 0
    overall_total = 0
    normal_correct = 0
    normal_total = 0
    abnormal_correct = 0
    abnormal_total = 0

    # Iterate over the test dataset
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Get predictions
        _, predicted = torch.max(outputs.data, 1)

        # Update accuracy counts
        overall_correct += (predicted == labels).sum().item()
        overall_total += labels.size(0)

        # Calculate accuracy for normal apples vs. abnormal apples
        normal_mask = labels == labels_dict['Normal_Apple']
        abnormal_mask = ~normal_mask
        normal_correct += (predicted[normal_mask] == labels[normal_mask]).sum().item()
        normal_total += normal_mask.sum().item()
        abnormal_correct += (predicted[abnormal_mask] == labels[abnormal_mask]).sum().item()
        abnormal_total += abnormal_mask.sum().item()

    # Calculate overall accuracy
    overall_accuracy = overall_correct / overall_total

    # Calculate accuracy for normal apples and abnormal apples separately
    normal_accuracy = normal_correct / normal_total if normal_total != 0 else 0.0
    abnormal_accuracy = abnormal_correct / abnormal_total if abnormal_total != 0 else 0.0

    # Print overall accuracy
    print(f"Overall accuracy: {overall_accuracy:.4f}")

    # Print accuracy for normal apples and abnormal apples separately
    print(f"Normal Apple accuracy: {normal_accuracy:.4f}")
    print(f"Abnormal Apple accuracy: {abnormal_accuracy:.4f}")



test_model(model, dataset_path)


Overall accuracy: 0.5333
Normal Apple accuracy: 0.9583
Abnormal Apple accuracy: 0.4271


In [25]:
# ... code for loading the model and test dataset ...

def test_model(model, datasetPath):
    model.eval()    

    # Load the test dataset
    dataset_path = datasetPath
    transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
    dataset = ImageFolder(dataset_path, transform=transform)
    test_dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

    labels_dict = dataset.class_to_idx

    # Track the overall test accuracy and accuracy by each type of apple
    overall_correct = 0
    overall_total = 0
    normal_correct = 0
    normal_total = 0
    abnormal_correct = 0
    abnormal_total = 0

    # Initialize the confusion matrix
    num_classes = len(labels_dict)
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)

    # Iterate over the test dataset
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Get predictions
        _, predicted = torch.max(outputs.data, 1)

        # Update accuracy counts
        overall_correct += (predicted == labels).sum().item()
        overall_total += labels.size(0)

        # Calculate accuracy for normal apples vs. abnormal apples
        normal_mask = labels == labels_dict['Normal_Apple']
        abnormal_mask = ~normal_mask
        normal_correct += (predicted[normal_mask] == labels[normal_mask]).sum().item()
        normal_total += normal_mask.sum().item()
        abnormal_correct += (predicted[abnormal_mask] == labels[abnormal_mask]).sum().item()
        abnormal_total += abnormal_mask.sum().item()

        # Update the confusion matrix
        for true_label, predicted_label in zip(labels.cpu().numpy(), predicted.cpu().numpy()):
            confusion_matrix[true_label][predicted_label] += 1

    # Calculate overall accuracy
    overall_accuracy = overall_correct / overall_total

    # Calculate accuracy for normal apples and abnormal apples separately
    normal_accuracy = normal_correct / normal_total if normal_total != 0 else 0.0
    abnormal_accuracy = abnormal_correct / abnormal_total if abnormal_total != 0 else 0.0

    # Print overall accuracy
    print(f"Overall accuracy: {overall_accuracy:.4f}")

    # Print accuracy for normal apples and abnormal apples separately
    print(f"Normal Apple accuracy: {normal_accuracy:.4f}")
    print(f"Abnormal Apple accuracy: {abnormal_accuracy:.4f}")

    # Print the confusion matrix
    print()
    print(labels_dict)
    print("Confusion Matrix:")
    print(confusion_matrix)

test_model(model, dataset_path)


Overall accuracy: 0.5333
Normal Apple accuracy: 0.9583
Abnormal Apple accuracy: 0.4271

{'Blotch_Apple': 0, 'Normal_Apple': 1, 'Rot_Apple': 2, 'Scab_Apple': 3}
Confusion Matrix:
[[11  8  1 10]
 [ 0 23  0  1]
 [ 0 21 11  6]
 [ 0  9  0 19]]


In [18]:

print(test_dataloader.class_to_idx)
# print(labels_dict)

AttributeError: 'DataLoader' object has no attribute 'class_to_idx'

In [13]:
# ... code for loading the model and test dataset ...

# Define the label dictionary
labels_dict = {
    'Blotch_Apple': 0,
    'Normal_Apple': 1,
    'Rot_Apple': 2,
    'Scab_Apple': 3
}

def test_model(model, dataloader):
    model.eval()

    # Track the overall test accuracy and accuracy by each type of apple
    overall_correct = 0
    overall_total = 0
    normal_correct = 0
    normal_total = 0
    abnormal_correct = 0
    abnormal_total = 0

    # Initialize the confusion matrix
    num_classes = len(labels_dict)
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)

    # Iterate over the test dataset
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # Get predictions
        _, predicted = torch.max(outputs.data, 1)

        # Update accuracy counts
        overall_correct += (predicted == labels).sum().item()
        overall_total += labels.size(0)

        # Calculate accuracy for normal apples vs. abnormal apples
        normal_mask = labels == labels_dict['Normal_Apple']
        abnormal_mask = ~normal_mask
        normal_correct += (predicted[normal_mask] == labels[normal_mask]).sum().item()
        normal_total += normal_mask.sum().item()
        abnormal_correct += (predicted[abnormal_mask] == labels[abnormal_mask]).sum().item()
        abnormal_total += abnormal_mask.sum().item()

        # Update the confusion matrix
        for true_label, predicted_label in zip(labels.cpu().numpy(), predicted.cpu().numpy()):
            confusion_matrix[true_label][predicted_label] += 1

    # Calculate overall accuracy
    overall_accuracy = overall_correct / overall_total

    # Calculate accuracy for normal apples and abnormal apples separately
    normal_accuracy = normal_correct / normal_total if normal_total != 0 else 0.0
    abnormal_accuracy = abnormal_correct / abnormal_total if abnormal_total != 0 else 0.0

    # Print overall accuracy
    print(f"Overall accuracy: {overall_accuracy:.4f}")

    # Print accuracy for normal apples and abnormal apples separately
    print(f"Normal Apple accuracy: {normal_accuracy:.4f}")
    print(f"Abnormal Apple accuracy: {abnormal_accuracy:.4f}")

    # Print the confusion matrix with labels
    print("Confusion Matrix:")
    print("True Labels \\ Predicted Labels:", end="\t")
    for i in range(num_classes):
        print(f"{i} ({list(labels_dict.keys())[i]})", end="\t")
    print()
    for i in range(num_classes):
        print(f"{i} ({list(labels_dict.keys())[i]}):", end="\t")
        for j in range(num_classes):
            print(confusion_matrix[i][j], end="\t")
        print()

test_model(model, test_dataloader)


Overall accuracy: 0.5333
Normal Apple accuracy: 0.9583
Abnormal Apple accuracy: 0.4271
Confusion Matrix:
True Labels \ Predicted Labels:	0 (Blotch_Apple)	1 (Normal_Apple)	2 (Rot_Apple)	3 (Scab_Apple)	
0 (Blotch_Apple):	11	8	1	10	
1 (Normal_Apple):	0	23	0	1	
2 (Rot_Apple):	0	21	11	6	
3 (Scab_Apple):	0	9	0	19	
